# Data from Semantic Kitti

In [ ]:
#| default_exp data

In [ ]:
#| export
import torch, os
import numpy as np
from pathlib import Path
from torch.utils.data import Dataset
from PIL import Image

In [ ]:
#| export
class Dataset(Dataset):
    def __init__(self, image_folder_path, mask_folder_path, transform=None):
        self.image_folder_path = image_folder_path
        self.mask_folder_path = mask_folder_path
        self.transform = transform
        self.image_filenames = os.listdir(image_folder_path)
        self.mask_filenames = os.listdir(mask_folder_path)

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder_path, self.image_filenames[idx])
        mask_name = os.path.join(self.mask_folder_path, self.mask_filenames[idx])

        image = Image.open(img_name)
        mask = Image.open(mask_name)

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask

In [ ]:
#| export
def merge_images(origin_path, merged_path):
    reflectance_path = origin_path/"reflectance"
    depth_path = origin_path/"depth"
    
    reflectance_files = os.listdir(reflectance_path)
    depth_files = os.listdir(depth_path)

    for reflectance_frame in reflectance_files:
        if reflectance_frame.endswith(".png"):

            reflectance_frame_path = os.path.join(reflectance_path, reflectance_frame)
            depth_frame_path = os.path.join(depth_path, reflectance_frame)


            reflectance_image = Image.open(reflectance_frame_path)
            depth_image = Image.open(depth_frame_path)


            reflectance_array = np.array(reflectance_image)
            depth_array = np.array(depth_image)

            merged_image = np.dstack((reflectance_array, depth_array))


            merged_image = Image.fromarray(merged_image)


            output_path = os.path.join(merged_path, reflectance_frame)
            merged_image.save(output_path)

Remap the classes for the chosen ones

0 - unknown/outlier: every other class

1 - car: 1

2 - truck: 4

3 - person/pedestrians: 6

4 - cyclist: 7, 2

5 - road: 9,

6 - sidewalk: 11

In [ ]:
#| export
def remap_segmentation_masks(origin_path, output_dir):
    
    input_dir = origin_path/"segmentation_mask"
    
    remapping_rules = {
      1: 1,
      4: 2,
      6: 3,
      7: 4,
      2: 4,
      9: 5,
      11: 6
    }
    
    for filename in os.listdir(input_dir):
      if filename.endswith(".png"):
          image_path = os.path.join(input_dir, filename)
          img = Image.open(image_path)
    
          img_array = np.array(img)
    
          remapped_array = np.zeros_like(img_array)
          for old_value, new_value in remapping_rules.items():
              remapped_array[img_array == old_value] = new_value
    
          remapped_img = Image.fromarray(remapped_array)
    
          output_path = os.path.join(output_dir, filename)
          remapped_img.save(output_path)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()